<a href="https://colab.research.google.com/github/nimishsoni/Transformers---HuggingFace/blob/main/Chatbot_Arena_MLE_Elo_Rating_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this notebook, we present data analysis on Chatbot Arena data collected from https://arena.lmsys.org between April 24 and Dec 6, 2023.

We explain different Elo calculation methods (online Elo and MLE Elo, also known as Bradley-Terry model) for model ranking.

To view the latest leaderboard, see https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard.


In [ ]:
from collections import defaultdict
import json, math, gdown
import numpy as np
import pandas as pd
import plotly.express as px
from tqdm import tqdm
pd.options.display.float_format = '{:.2f}'.format

# Obtaining and Cleaning the Tournament Data
We are hosting the initial tournament results as a JSON file on Google Drive. We use the `gdown` function to download the data. The data contains all the battels and voting results collected for ranking models.

In [ ]:
url = "https://drive.google.com/file/d/1vv_-tI_hGIJSraHUGRz_9P8ip3qUCD9E/view"
filename = gdown.download(url, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1vv_-tI_hGIJSraHUGRz_9P8ip3qUCD9E
To: /content/clean_battle_anony_20231206.json
100%|██████████| 27.5M/27.5M [00:00<00:00, 71.2MB/s]


In [ ]:
battles = pd.read_json(filename).sort_values(ascending=True, by=["tstamp"])
battles

,model_a,model_b,winner,judge,turn,anony,language,tstamp
0,chatglm-6b,koala-13b,model_b,arena_user_964,1,True,English,1682351591.13
1,oasst-pythia-12b,alpaca-13b,tie,arena_user_964,1,True,English,1682351654.67
2,koala-13b,oasst-pythia-12b,model_b,arena_user_964,1,True,English,1682351708.94
3,vicuna-13b,oasst-pythia-12b,model_b,arena_user_964,1,True,English,1682351785.19
4,vicuna-13b,koala-13b,model_a,arena_user_964,1,True,English,1682351891.66
...,...,...,...,...,...,...,...,...
136164,pplx-70b-online,gpt-4-0314,tie (bothbad),arena_user_65599,1,True,English,1701913006.82
136165,gpt-4-turbo,gpt-4-0314,model_b,arena_user_35892,1,True,English,1701913045.32
136166,pplx-70b-online,gpt-4-0314,tie (bothbad),arena_user_65572,2,True,English,1701913108.02
136167,wizardlm-70b,claude-instant-1,model_b,arena_user_65604,1,True,English,1701913124.05


# Exploratory Analysis

Before computing the Elo ratings, we first conduct some basic exploratory analysis to highlight a few key properties and caveates with this data.

## Statistics

We allowed the user to declare a tie between the pairs of models.  To collect additional data, later in the tournament we also allowed the user to declare a tie in which both models were bad.  There were a significant portion of tied outcomes.

In [ ]:
fig = px.bar(battles["winner"].value_counts(),
             title="Counts of Battle Outcomes", text_auto=True, height=400)
fig.update_layout(xaxis_title="Battle Outcome", yaxis_title="Count",
                  showlegend=False)
fig

In [ ]:
battles_no_ties = battles[~battles["winner"].str.contains("tie")]

## Non-uniform Model Frequency

The model frequency is not uniform because of the follwoing reasons:
- Several different matching and sampling algorithms were used. We employed uniform sampling as well as weighted sampling methods, which assign greater weights to better models.
- Some new models were added later.


In [ ]:
fig = px.bar(pd.concat([battles["model_a"], battles["model_b"]]).value_counts(),
             title="Battle Count for Each Model", text_auto=True)
fig.update_layout(xaxis_title="model", yaxis_title="Battle Count", height=400,
                  showlegend=False)
fig

We examing the number of pairings for each combination of models.

In [ ]:
def visualize_battle_count(battles, title, show_num_models=30):
    ptbl = pd.pivot_table(battles, index="model_a", columns="model_b", aggfunc="size",
                          fill_value=0)
    battle_counts = ptbl + ptbl.T
    ordering = battle_counts.sum().sort_values(ascending=False).index
    ordering = ordering[:show_num_models]
    fig = px.imshow(battle_counts.loc[ordering, ordering],
                    title=title, text_auto=True)
    fig.update_layout(xaxis_title="Model B",
                      yaxis_title="Model A",
                      xaxis_side="top", height=800, width=800,
                      title_y=0.07, title_x=0.5,
                      font=dict(size=10))
    fig.update_traces(hovertemplate=
                      "Model A: %{y}<br>Model B: %{x}<br>Count: %{z}<extra></extra>")
    return fig

fig = visualize_battle_count(battles, title="Battle Count of Each Combination of Models", show_num_models=30)
fig

### Battles Excluding Ties

In [ ]:
visualize_battle_count(battles_no_ties, "Battle Count for Each Combination of Models (without Ties)")

### Counting Ties

In [ ]:
visualize_battle_count(battles[battles['winner'].str.contains("tie")], "Tie Count for Each Combination of Models")

## Inferred Language

We also inferred the language for each conversation using `polyglot` package. This is just an estimate but will help guide future analysis.  The vast majority of conversations were in English.

In [ ]:
topk = 15
fig = px.bar(battles["language"].value_counts().head(topk),
             title=f"Battle Counts for the Top {topk} Languages",
             text_auto=True, height=400)
fig.update_layout(xaxis_title="Language", yaxis_title="Count", showlegend=False)
fig

## Number of Conversation Turns

We also noticed that most counversations only have one turn.

In [ ]:
fig = px.histogram(battles["turn"],
             title=f"Number of Conversation Turns",
             text_auto=True, height=400)
fig.update_layout(xaxis_title="Turns", yaxis_title="Count", showlegend=False)
fig

## Pairwise Win Fractions

Finally, we can also compute the pairwise win fractions. However, because each model can play as Model A and as Model B and win in both situations we need to compute the wins in both configurations divided by the number of pairings of each model.

In [ ]:
def compute_pairwise_win_fraction(battles, max_num_models=30):
    # Times each model wins as Model A
    a_win_ptbl = pd.pivot_table(
        battles[battles['winner'] == "model_a"],
        index="model_a", columns="model_b", aggfunc="size", fill_value=0)

    # Table counting times each model wins as Model B
    b_win_ptbl = pd.pivot_table(
        battles[battles['winner'] == "model_b"],
        index="model_a", columns="model_b", aggfunc="size", fill_value=0)

    # Table counting number of A-B pairs
    num_battles_ptbl = pd.pivot_table(battles,
        index="model_a", columns="model_b", aggfunc="size", fill_value=0)

    # Computing the proportion of wins for each model as A and as B
    # against all other models
    row_beats_col_freq = (
        (a_win_ptbl + b_win_ptbl.T) /
        (num_battles_ptbl + num_battles_ptbl.T)
    )

    # Arrange ordering according to proprition of wins
    prop_wins = row_beats_col_freq.mean(axis=1).sort_values(ascending=False)
    prop_wins = prop_wins[:max_num_models]
    model_names = list(prop_wins.keys())
    row_beats_col = row_beats_col_freq.loc[model_names, model_names]
    return row_beats_col

def visualize_pairwise_win_fraction(battles, title, max_num_models=30):
    row_beats_col = compute_pairwise_win_fraction(battles, max_num_models)
    fig = px.imshow(row_beats_col, color_continuous_scale='RdBu',
                    text_auto=".2f", title=title)
    fig.update_layout(xaxis_title=" Model B: Loser",
                  yaxis_title="Model A: Winner",
                  xaxis_side="top", height=900, width=900,
                  title_y=0.07, title_x=0.5)
    fig.update_traces(hovertemplate=
                  "Model A: %{y}<br>Model B: %{x}<br>Fraction of A Wins: %{z}<extra></extra>")

    return fig

In [ ]:
fig = visualize_pairwise_win_fraction(battles_no_ties,
      title = "Fraction of Model A Wins for All Non-tied A vs. B Battles")
fig

## Preliminary Ranking

Using just the average win rate against all other models we can already compute an estimated leaderboard.
However, this method may not be as scalable as the Elo rating system that we will use later because this method requires data from all model combinations.

In [ ]:
row_beats_col_freq = compute_pairwise_win_fraction(battles_no_ties)
fig = px.bar(row_beats_col_freq.mean(axis=1).sort_values(ascending=False),
             title="Average Win Rate Against All Other Models (Assuming Uniform Sampling and No Ties)",
             text_auto=".2f")
fig.update_layout(yaxis_title="Average Win Rate", xaxis_title="Model",
                  showlegend=False)
fig

#Elo Ratings

The [Elo rating system ](https://en.wikipedia.org/wiki/Elo_rating_system)is a method for calculating the relative skill levels of players, which has been widely adopted in chess and other competitive games. The difference in the ratings between two players serves as a predictor of the outcome of a match. The Elo rating system works well for our case because we have multiple models and we run pairwise battles between them.
In this section, we present different methods for calculating Elo ratings.

### Compute Ratings
We first use the online linear update algorithm to compute Elo ratings.
We choose a small K-factor of 4 to make the Elo ratings more stable and less biased towards recent games.

In [ ]:
def compute_online_elo(battles, K=4, SCALE=400, BASE=10, INIT_RATING=1000):
    rating = defaultdict(lambda: INIT_RATING)

    for rd, model_a, model_b, winner in battles[['model_a', 'model_b', 'winner']].itertuples():
        ra = rating[model_a]
        rb = rating[model_b]
        ea = 1 / (1 + BASE ** ((rb - ra) / SCALE))
        eb = 1 / (1 + BASE ** ((ra - rb) / SCALE))
        if winner == "model_a":
            sa = 1
        elif winner == "model_b":
            sa = 0
        elif winner == "tie" or winner == "tie (bothbad)":
            sa = 0.5
        else:
            raise Exception(f"unexpected vote {winner}")
        rating[model_a] += K * (sa - ea)
        rating[model_b] += K * (1 - sa - eb)

    # calibrate llama-13b to 800
    delta = (800-rating["llama-13b"])
    for model in battles["model_a"].unique():
        rating[model] += delta

    return rating

In [ ]:
def preety_print_model_ratings(ratings):
    df = pd.DataFrame([
        [n, ratings[n]] for n in ratings.keys()
    ], columns=["Model", "Elo rating"]).sort_values("Elo rating", ascending=False).reset_index(drop=True)
    df["Elo rating"] = (df["Elo rating"] + 0.5).astype(int)
    df.index = df.index + 1
    return df

online_elo_ratings = compute_online_elo(battles)
preety_print_model_ratings(online_elo_ratings)

,Model,Elo rating
1,gpt-4-turbo,1150
2,claude-1,1088
3,gpt-4-0314,1075
4,gpt-4-0613,1050
5,gpt-3.5-turbo-0314,1046
6,claude-2.0,1036
7,gpt-3.5-turbo-0613,1030
8,claude-2.1,1019
9,tulu-2-dpo-70b,1010
10,wizardlm-70b,1007


However, even with a small K-factor, we still found this online update algorithm to be unstable.

To demonstrate it, we recompute Elo rating by using the reversed game order and observe significant difference due to online update of Elo which biases the recent games.

In [ ]:
def preety_print_two_ratings(ratings_1, ratings_2, column_names):
    df = pd.DataFrame([
        [n, ratings_1[n], ratings_2[n]] for n in ratings_1.keys()
    ], columns=["Model", column_names[0], column_names[1]]).sort_values(column_names[0], ascending=False).reset_index(drop=True)
    df[column_names[0]] = (df[column_names[0]] + 0.5).astype(int)
    df[column_names[1]] = (df[column_names[1]] + 0.5).astype(int)
    df.index = df.index + 1
    return df

elo_mle_ratings_reverse = compute_online_elo(battles.iloc[::-1])
preety_print_two_ratings(online_elo_ratings,
                         elo_mle_ratings_reverse,
                         column_names=["Elo rating", "Elo rating with reverse order"])

,Model,Elo rating,Elo rating with reverse order
1,gpt-4-turbo,1150,1178
2,claude-1,1088,1194
3,gpt-4-0314,1075,1262
4,gpt-4-0613,1050,1193
5,gpt-3.5-turbo-0314,1046,1143
6,claude-2.0,1036,1139
7,gpt-3.5-turbo-0613,1030,1132
8,claude-2.1,1019,1033
9,tulu-2-dpo-70b,1010,1023
10,wizardlm-70b,1007,1046



### Maximum Likelihood Estimation for Elo Ratings (aka [Bradley-Terry model](https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model))

In the context of LLM evaluation, models can be assumed to be static. In this case, we can directly fit the ratings by maximum likelihood estimation method (aka Bradley-Terry model), which produce significantly stable ratings. Here we provide an implementation with logistic regression.

In [ ]:
def compute_mle_elo(df, SCALE=400, BASE=10, INIT_RATING=1000):
    from sklearn.linear_model import LogisticRegression
    models = pd.concat([df["model_a"], df["model_b"]]).unique()
    models = pd.Series(np.arange(len(models)), index=models)

    # duplicate battles
    df = pd.concat([df, df], ignore_index=True)
    p = len(models.index)
    n = df.shape[0]

    X = np.zeros([n, p])
    X[np.arange(n), models[df["model_a"]]] = +math.log(BASE)
    X[np.arange(n), models[df["model_b"]]] = -math.log(BASE)

    # one A win => two A win
    Y = np.zeros(n)
    Y[df["winner"] == "model_a"] = 1.0

    # one tie => one A win + one B win
    # find tie + tie (both bad) index
    tie_idx = (df["winner"] == "tie") | (df["winner"] == "tie (bothbad)")
    tie_idx[len(tie_idx)//2:] = False
    Y[tie_idx] = 1.0

    lr = LogisticRegression(fit_intercept=False)
    lr.fit(X,Y)

    elo_scores = SCALE * lr.coef_[0] + INIT_RATING

    # calibrate llama-13b to 800
    elo_scores += (800-elo_scores[models["llama-13b"]])
    return pd.Series(elo_scores, index = models.index).sort_values(ascending=False)

In [ ]:
elo_mle_ratings = compute_mle_elo(battles)
preety_print_model_ratings(elo_mle_ratings)

,Model,Elo rating
1,gpt-4-turbo,1217
2,gpt-4-0314,1201
3,claude-1,1153
4,gpt-4-0613,1152
5,claude-2.0,1127
6,claude-2.1,1118
7,gpt-3.5-turbo-0613,1112
8,claude-instant-1,1109
9,gpt-3.5-turbo-0314,1105
10,tulu-2-dpo-70b,1105


### Compute Bootstrap Confidence Interavals for MLE Elo Scores

We can further use bootstrap to estimate the confidence intervals as well.


In [ ]:
def get_bootstrap_result(battles, func_compute_elo, num_round):
    rows = []
    for i in tqdm(range(num_round), desc="bootstrap"):
        rows.append(func_compute_elo(battles.sample(frac=1.0, replace=True)))
    df = pd.DataFrame(rows)
    return df[df.median().sort_values(ascending=False).index]


In [ ]:
BOOTSTRAP_ROUNDS = 100

np.random.seed(42)
bootstrap_elo_lu = get_bootstrap_result(battles, compute_mle_elo, BOOTSTRAP_ROUNDS)

bootstrap: 100%|██████████| 100/100 [04:30<00:00,  2.70s/it]


In [ ]:
def visualize_bootstrap_scores(df, title):
    bars = pd.DataFrame(dict(
        lower = df.quantile(.025),
        rating = df.quantile(.5),
        upper = df.quantile(.975))).reset_index(names="model").sort_values("rating", ascending=False)
    bars['error_y'] = bars['upper'] - bars["rating"]
    bars['error_y_minus'] = bars['rating'] - bars["lower"]
    bars['rating_rounded'] = np.round(bars['rating'], 2)
    fig = px.scatter(bars, x="model", y="rating", error_y="error_y",
                     error_y_minus="error_y_minus", text="rating_rounded",
                     title=title)
    fig.update_layout(xaxis_title="Model", yaxis_title="Rating",
                      height=600)
    return fig

fig = visualize_bootstrap_scores(bootstrap_elo_lu, "Bootstrap of MLE Elo Rating Estimates")
fig

We previously apply bootstrapping on the online Elo to obtain stabler ratings.

In [ ]:
np.random.seed(42)
bootstrap_online_elo = get_bootstrap_result(battles, compute_online_elo, BOOTSTRAP_ROUNDS)

bootstrap: 100%|██████████| 100/100 [01:06<00:00,  1.51it/s]


We can see the bootstrapping medians obtained by both methods are similar.

In [ ]:
preety_print_two_ratings(bootstrap_elo_lu.quantile(.5),
                         bootstrap_online_elo.quantile(.5),
                         column_names=["Bootstrap Median of MLE Elo", "Bootstrap Median of Online Elo"])

,Model,Bootstrap Median of MLE Elo,Bootstrap Median of Online Elo
1,gpt-4-turbo,1217,1220
2,gpt-4-0314,1201,1204
3,gpt-4-0613,1153,1156
4,claude-1,1152,1158
5,claude-2.0,1128,1130
6,claude-2.1,1119,1120
7,gpt-3.5-turbo-0613,1112,1115
8,claude-instant-1,1109,1108
9,gpt-3.5-turbo-0314,1106,1104
10,tulu-2-dpo-70b,1106,1106


However, online Elo's confidence intervals are significantly larger than the MLE Elo.

In [ ]:
fig = visualize_bootstrap_scores(bootstrap_online_elo, "Bootstrap of Online Elo Rating Estimates")
fig

### Predict Win Rates
Utilizing Elo ratings allows us to predict win probabilities. By comparing the predicted win rates with the actual win rates, we can gain insight into the accuracy and quality of the Elo rating system.






In [ ]:
def predict_win_rate(elo_ratings, SCALE=400, BASE=10, INIT_RATING=1000):
    names = sorted(list(elo_ratings.keys()))
    wins = defaultdict(lambda: defaultdict(lambda: 0))
    for a in names:
        for b in names:
            ea = 1 / (1 + BASE ** ((elo_ratings[b] - elo_ratings[a]) / SCALE))
            wins[a][b] = ea
            wins[b][a] = 1 - ea

    data = {
        a: [wins[a][b] if a != b else np.NAN for b in names]
        for a in names
    }

    df = pd.DataFrame(data, index=names)
    df.index.name = "model_a"
    df.columns.name = "model_b"
    return df.T

In [ ]:
win_rate = predict_win_rate(dict(bootstrap_elo_lu.quantile(0.5)))
ordered_models = win_rate.mean(axis=1).sort_values(ascending=False).index
ordered_models = ordered_models[:30]
fig = px.imshow(win_rate.loc[ordered_models, ordered_models],
                color_continuous_scale='RdBu', text_auto=".2f",
                title="Predicted Win Rate Using Elo Ratings for Model A in an A vs. B Battle")
fig.update_layout(xaxis_title="Model B",
                  yaxis_title="Model A",
                  xaxis_side="top", height=900, width=900,
                  title_y=0.07, title_x=0.5)
fig.update_traces(hovertemplate=
                  "Model A: %{y}<br>Model B: %{x}<br>Win Rate: %{z}<extra></extra>")
fig

### Compute Bootstrap Confidence Intervals Assuming Uniform Sampling

We also study how the ratings will change if we only sample an equal number of battles for each model pair.

In [ ]:
def sample_battle_even(battles, n_per_battle):
    groups = battles.groupby(["model_a", "model_b"], as_index=False)
    resampled = (groups
                 .apply(lambda grp: grp.sample(n_per_battle, replace=True))
                 .reset_index(drop=True))
    return resampled

In [ ]:
num_samples = 50
battles_even = sample_battle_even(battles, num_samples)
pd.pivot_table(battles_even, index="model_a", columns="model_b", aggfunc="size", fill_value=0)

model_b,RWKV-4-Raven-14B,alpaca-13b,chatglm-6b,chatglm2-6b,chatglm3-6b,claude-1,claude-2.0,claude-2.1,claude-instant-1,codellama-34b-instruct,...,starling-lm-7b-alpha,tulu-2-dpo-70b,vicuna-13b,vicuna-33b,vicuna-7b,wizardlm-13b,wizardlm-70b,yi-34b-chat,zephyr-7b-alpha,zephyr-7b-beta
model_a,,,,,,,,,,,,,,,,,,,,,
RWKV-4-Raven-14B,0,50,50,0,0,50,50,0,50,0,...,0,0,50,50,50,50,0,0,0,0
alpaca-13b,50,0,50,0,0,50,50,0,50,0,...,0,0,50,50,50,50,0,0,0,0
chatglm-6b,50,50,0,0,0,50,50,0,50,0,...,0,0,50,50,50,50,0,0,0,0
chatglm2-6b,0,50,0,0,50,50,50,50,50,50,...,0,50,50,50,50,50,50,50,50,50
chatglm3-6b,0,0,0,50,0,50,50,50,50,50,...,50,50,50,50,50,50,50,50,0,50
claude-1,50,50,50,50,50,0,50,50,50,50,...,50,50,50,50,50,50,50,50,50,50
claude-2.0,50,50,50,50,50,50,0,50,50,50,...,50,50,50,50,50,50,50,50,50,50
claude-2.1,0,0,0,50,50,50,50,0,50,50,...,50,50,50,50,50,50,50,50,0,50
claude-instant-1,50,50,50,50,50,50,50,50,0,50,...,50,50,50,50,50,50,50,50,50,50


In [ ]:
# Sampling Battles Evenly
def get_bootstrap_even_sample(battles, n_per_battle, func_compute_elo, num_round=BOOTSTRAP_ROUNDS):
    rows = []
    for n in tqdm(range(num_round), desc="sampling battles evenly"):
        resampled = sample_battle_even(battles, n_per_battle)
        rows.append(func_compute_elo(resampled))
    df = pd.DataFrame(rows)
    return df[df.median().sort_values(ascending=False).index]

In [ ]:
print("number of samples per battle pair:", num_samples)
bootstrap_even_lu = get_bootstrap_even_sample(battles, num_samples, compute_mle_elo, num_round=100)

number of samples per battle pair: 50


sampling battles evenly: 100%|██████████| 100/100 [03:44<00:00,  2.24s/it]


In [ ]:
fig = visualize_bootstrap_scores(bootstrap_even_lu, f"Bootstrap of MLE Elo Estimates - Even sample")
fig

# Language-specific Leaderboards
We present two language-specific leaderboards, by isolating the chat data into two subsets based on the language: (1) English-only and (2) Non-English.

## English-only

In [ ]:
english_only_battles = battles[battles["language"] == "English"]
elo_ratings = compute_mle_elo(english_only_battles)
compute_mle_elo(elo_ratings)

,Model,Elo rating
1,gpt-4-turbo,1222
2,gpt-4-0314,1201
3,gpt-4-0613,1155
4,claude-1,1153
5,claude-2.0,1133
6,claude-2.1,1117
7,yi-34b-chat,1116
8,claude-instant-1,1114
9,tulu-2-dpo-70b,1114
10,gpt-3.5-turbo-0613,1113


## Non-English

In [ ]:
non_english_battles = battles[battles["language"] != "English"]
elo_ratings = compute_mle_elo(non_english_battles)
compute_mle_elo(elo_ratings)

,Model,Elo rating
1,gpt-4-0314,1205
2,gpt-4-turbo,1199
3,claude-1,1155
4,gpt-4-0613,1139
5,gpt-3.5-turbo-0314,1127
6,claude-2.1,1119
7,claude-2.0,1107
8,gpt-3.5-turbo-0613,1104
9,claude-instant-1,1087
10,tulu-2-dpo-70b,1071


# Links



Some good resources to learn more about Elo rating systems:
- Elo rating system https://en.wikipedia.org/wiki/Elo_rating_system
- Bradley-Terry model https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model
- An introduction video https://www.youtube.com/watch?v=AsYfbmp0To0
- A FiveThirtyEight article https://fivethirtyeight.com/methodology/how-our-nfl-predictions-work/
